In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import seaborn as sns
import pandas as pd
import gensim
import nltk
import re
import string
from curses.ascii import isprint

In [3]:
stop_words = set(open('stop_words').read().split('\n')[:-1])
#stop_words = 'the,for,to,you,call,that'.split(',')
#print stop_words

In [4]:
emails_table = pd.read_csv("data/emails/Emails.csv").dropna()
person_table = pd.read_csv("data/emails/Persons.csv").dropna()
alias_table = pd.read_csv("data/emails/Aliases.csv").dropna()
email_rec_table = pd.read_csv("data/emails/EmailReceivers.csv").dropna()

In [5]:
emails_table.head()

,Id,DocNumber,MetadataSubject,MetadataTo,MetadataFrom,SenderPersonId,MetadataDateSent,MetadataDateReleased,MetadataPdfLink,MetadataCaseNumber,...,ExtractedTo,ExtractedFrom,ExtractedCc,ExtractedDateSent,ExtractedCaseNumber,ExtractedDocNumber,ExtractedDateReleased,ExtractedReleaseInPartOrFull,ExtractedBodyText,RawText
2,3,C05739547,CHRIS STEVENS,;H,"Mills, Cheryl D",32.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739547...,F-2015-04841,...,B6,"Mills, Cheryl D <MillsCD@state.gov>","Abedin, Huma","Wednesday, September 12, 2012 11:52 AM",F-2015-04841,C05739547,05/14/2015,RELEASE IN PART,Thx,UNCLASSIFIED\nU.S. Department of State\nCase N...
16,17,C05739573,THE YOUTH OF LIBYA,H,"Sherman, Wendy R",213.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739573...,F-2015-04841,...,"H; Mills, Cheryl D; Sullivan, Jacob J; Nuiand,...","Sherman, Wendy R <ShermanWR@state.gov>","Escrogirna, Ana A; Grantham, Chris W","Wednesday, September 12, 2012 9:57 PM",F-2015-04841,C05739573,05/13/2015,RELEASE IN FULL,Hope. See picture below Kamala sent.,"-ottgat,t\n4 44 4, 0\n.\n1.3.141.4443\nUNCLASS..."
17,18,C05739575,ONE MORE PHOTO,H,"Sherman, Wendy R",213.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739575...,F-2015-04841,...,"H; Mills, Cheryl D; Sullivan, Jacob J, Nuland,...","Sherman, Wendy R <ShermanWR@state.gov>","Escrogima, Ana A","Wednesday, September 12, 2012 10:03 PM",F-2015-04841,C05739575,05/13/2015,RELEASE IN FULL,Another photo.,c. 371.14\nUNCLASSIFIED\nU.S. Department of St...
65,66,C05739638,BRITS AND LIBYA,sullivanjj@state.gov,H,80.0,2011-04-02T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739638...,F-2015-04841,...,'sullivanjj@state.gov',H <hrod17@clintonemailcom>,Huma Abedin,"Saturday, April 02, 2011 1034 AM",F-2015-04841,C05739638,05/13/2015,RELEASE IN PART,"When is Chris Stevens going?\n---,",UNCLASSIFIED\nU.S. Department of State\nCase N...
82,83,C05739656,THE CABLE: ROMNEY TO COMPARE BENGHAZI TO ORIGI...,millscd@state.gov,H,80.0,2012-10-08T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH3/DOC_0C05739656...,F-2015-04841,...,"'miliscd@stategov""",H <hrod17@clintonemail.com>,"Reines, Philippe 1; Mills, Cheryl D; Adams, Da...","Monday, October 08, 2012 07:26 AM",F-2015-04841,C05739656,05/13/2015,RELEASE IN FULL,Can you call me?,UNCLASSIFIED\nU.S. Department of State\nCase N...


In [6]:
person_table.head()

,Id,Name
0,1,111th Congress
1,2,AGNA USEMB Kabul Afghanistan
2,3,AP
3,4,ASUNCION
4,5,Alec


In [7]:
alias_table.head()

,Id,Alias,PersonId
0,1,111th congress,1
1,2,agna usemb kabul afghanistan,2
2,3,ap,3
3,4,asuncion,4
4,5,alec,5


In [8]:
email_rec_table.head()

,Id,EmailId,PersonId
0,1,1,80
1,2,2,80
2,3,3,228
3,4,3,80
4,5,4,80


In [9]:
print len(emails_table)
print len(person_table)
print len(alias_table)
print len(email_rec_table)

908
513
850
9306


In [10]:
emails_table['doc'] = emails_table["ExtractedSubject"] + " " + emails_table["ExtractedBodyText"]

In [11]:
def clean_text(text):
    print
    print text
    # Remove any beginning or ending whitespace
    text.strip()
    # Combine conjunctions
    text = re.sub(r"'", " ", text)
    # Replace punctuation with space
    text = text.translate(string.maketrans(string.punctuation,''.join(" " for char in string.punctuation)))
    # Replace all control characters
    #text = ''.join(char for char in text if isprint(char))
    #text = re.sub(r"\n", " ", text)
    # Remove Digits
    text = re.sub(r"[0-9]", " ", text)
    
    
    text = text.lower()
    
    text_list = [token for token in text.split() if token not in stop_words and len(token) > 1]
    print text_list
    print 
    
    return text_list

In [12]:
emails_table['token_list'] = emails_table['doc'].apply(lambda s: clean_text(s))


Re: Chris Stevens Thx
['chris', 'stevens']


Fw: The Youth of Libya Hope. See picture below Kamala sent.
['youth', 'libya', 'hope', 'picture', 'below', 'kamala']


Fw: One More Photo Another photo.
['photo', 'photo']


Re: Brits and Libya When is Chris Stevens going?
---,
['brits', 'libya', 'chris', 'stevens']


Re: The Cable: Romney to compare Benghazi to original 9/11 attacks Can you call me?
['cable', 'romney', 'compare', 'benghazi', 'original', 'attacks']


Re: Question and update --I approved. Will it be picked up?
--call w King should be scheduled so keep working w Huma
--I will speak to Holder. Does he need to speak secure? I can do either.
['question', 'approved', 'picked', 'king', 'scheduled', 'huma', 'speak', 'holder', 'speak', 'secure']


Re: Speech draft for Friday at CSIS We will send a printed copy to you with the book tonight. Should be there shortly.
['speech', 'draft', 'friday', 'csis', 'send', 'printed', 'copy', 'book', 'tonight', 'shortly']


Re: PDB Yes, II give me

In [26]:
dictionary = gensim.corpora.Dictionary(list(emails_table['token_list']))
dictionary.save('models/dictionary.dict')

In [27]:
corpus = [dictionary.doc2bow(text) for text in emails_table['token_list']]
gensim.corpora.MmCorpus.serialize('models/corpus.mm', corpus) 

In [28]:
lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=20)
lda.save('models/emails_lda.model')

In [17]:
import pyLDAvis.gensim as gensimvis
import pyLDAvis

In [18]:
vis_data = gensimvis.prepare(lda, corpus, dictionary, R=10)
pyLDAvis.display(vis_data)

In [19]:
hdp = gensim.models.hdpmodel.HdpModel(corpus, dictionary, T=50)                                  
hdp.save('models/emails_hdp.model')

In [20]:
alpha, beta = hdp.hdp_to_lda()
hdp_lda = gensim.models.LdaModel(id2word=hdp.id2word,num_topics=len(alpha), alpha=alpha, eta=hdp.m_eta) 
hdp_lda.expElogbeta = numpy.array(beta, dtype=numpy.float32) 

In [21]:
vis_data = gensimvis.prepare(hdp_lda, corpus, dictionary, R=10)
pyLDAvis.display(vis_data)

In [22]:
s = 'a'

In [23]:
s.join(" ")

' '